### This file merges variable name file with shaply file, and variable value and crisis dummy

### a. and add base and final rows to each country and year

### b. also ***(vertically)*** merges with metadata (country group, year group) using their group averages

### c. merge with crisis dummy files

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib

# load and merge
data_dir = 'Q:\DATA\SPRAIRU\Tableau visualization\datasets'

df_shap = pd.read_excel(os.path.join(data_dir, 'shapvalues_norm_full.xlsx'))
df_varname = pd.read_excel(os.path.join(data_dir, 'mapping variable labels.xlsx'))
df_country_type = pd.read_excel(os.path.join(data_dir, 'mapping country group.xlsx'))
df_value = pd.read_excel(os.path.join(data_dir, 'variable_full.xlsx'))
df_crisis = pd.read_excel(os.path.join(data_dir, 'mapping precrisis.xlsx'))


df_shap.drop(columns=['avg','norm_shap'], inplace= True)

#### merge shap with varname

In [2]:
df_new = pd.merge(df_shap, df_varname, left_index= True, right_on= 'Variable', how= 'left')
df_new.reset_index(inplace= True)
df_new.tail()

,index,year,country,shap,Variable,Label
218278,0,2014,Romania,0.867583,snig4,"Corp Debt, Sub IG"
218279,0,2015,Romania,0.807161,snig4,"Corp Debt, Sub IG"
218280,0,2016,Romania,0.876883,snig4,"Corp Debt, Sub IG"
218281,0,2017,Romania,0.995385,snig4,"Corp Debt, Sub IG"
218282,0,2018,Romania,1.251693,snig4,"Corp Debt, Sub IG"


#### reshaping to add base and total as new variables

In [3]:
# reshape to wide
df_new = df_new[['year','country','Label','shap']]

df_new = pd.pivot_table(df_new, values ='shap', index =['country','year'], columns = 'Label')

df_new.reset_index(inplace= True)

In [4]:
# add base and total rows
df_new['base'] = 50

df_new['total']=df_new[df_new.columns.tolist()[2:]].apply(sum, axis = 1)

df_new['total'].plot.hist()

In [5]:
# reshape to long
df_new = df_new.melt(id_vars=['country','year'], value_vars=df_new.columns.tolist()[2:], var_name='Variable', value_name='shap')
df_new.head()

,country,year,Variable,shap
0,"Afghanistan, I.S. of",1980,3-month T-Bill rate,2.851679
1,"Afghanistan, I.S. of",1981,3-month T-Bill rate,2.882570
2,"Afghanistan, I.S. of",1982,3-month T-Bill rate,-0.553009
3,"Afghanistan, I.S. of",1983,3-month T-Bill rate,-0.022255
4,"Afghanistan, I.S. of",1984,3-month T-Bill rate,-0.437330


#### aggregating and concatnate vertically (to add country & year group averages rows)

In [6]:
# add column for "variable value" from file
df_value = df_value[['country','year','Variable','variable value']]
df_value.head()

,country,year,Variable,variable value
0,"Afghanistan, I.S. of",1980,3-month T-Bill rate,0.000000
1,"Afghanistan, I.S. of",1981,3-month T-Bill rate,0.000000
2,"Afghanistan, I.S. of",1982,3-month T-Bill rate,11.100833
3,"Afghanistan, I.S. of",1983,3-month T-Bill rate,8.944167
4,"Afghanistan, I.S. of",1984,3-month T-Bill rate,9.897500


In [7]:
df_value.loc[df_value['variable value'] == 0, 'variable value'] = np.nan # set 0 to nan
df_new = pd.merge(df_new, df_value, left_on = ['country','year','Variable'], right_on = ['country','year','Variable'], how= 'left')

In [8]:
# merge with country group
df_group = pd.merge(df_new, df_country_type, left_on= 'country', right_on= 'country', how = 'left')
df_group.reset_index(inplace = True)

df_group.head()

,index,country,year,Variable,shap,variable value,Oil exporters,Income
0,0,"Afghanistan, I.S. of",1980,3-month T-Bill rate,2.851679,NaN,0,LIC
1,1,"Afghanistan, I.S. of",1981,3-month T-Bill rate,2.882570,NaN,0,LIC
2,2,"Afghanistan, I.S. of",1982,3-month T-Bill rate,-0.553009,11.100833,0,LIC
3,3,"Afghanistan, I.S. of",1983,3-month T-Bill rate,-0.022255,8.944167,0,LIC
4,4,"Afghanistan, I.S. of",1984,3-month T-Bill rate,-0.437330,9.897500,0,LIC


In [9]:
# aggregate by country income group
df_income_group = df_group.groupby(['Income','year','Variable']).agg({'shap':'mean', 'variable value': 'mean'})
df_income_group.reset_index(inplace= True)

df_income_group.rename({'Income' : 'country'}, axis = 1, inplace= True)
df_income_group.head()

,country,year,Variable,shap,variable value
0,AE,1980,3-month T-Bill rate,2.502182,NaN
1,AE,1980,"Bank PD, average",-0.318967,NaN
2,AE,1980,Banking Claims to GDP,-2.134330,NaN
3,AE,1980,Cap Adeq Ratio,-0.958507,NaN
4,AE,1980,"Corp Debt, Sub IG",1.769560,NaN


In [10]:
# aggregate by oil group
df_oil_group = df_group.groupby(['Oil exporters','year','Variable']).agg({'shap':'mean', 'variable value': 'mean'})
df_oil_group.reset_index(inplace= True)
df_oil_group.rename({'Oil exporters' : 'country'}, axis = 1, inplace= True)

df_oil_group.country = df_oil_group.country.apply(lambda x: 'oil-exporter' if x == 1 else 'non-oil exporter')
df_oil_group.head()

## append country group averages to original table
df_full = pd.concat([df_group, df_income_group, df_oil_group], axis = 0, sort = True)

In [11]:
df_group.head()

,index,country,year,Variable,shap,variable value,Oil exporters,Income
0,0,"Afghanistan, I.S. of",1980,3-month T-Bill rate,2.851679,NaN,0,LIC
1,1,"Afghanistan, I.S. of",1981,3-month T-Bill rate,2.882570,NaN,0,LIC
2,2,"Afghanistan, I.S. of",1982,3-month T-Bill rate,-0.553009,11.100833,0,LIC
3,3,"Afghanistan, I.S. of",1983,3-month T-Bill rate,-0.022255,8.944167,0,LIC
4,4,"Afghanistan, I.S. of",1984,3-month T-Bill rate,-0.437330,9.897500,0,LIC


In [12]:
# merge with year group
year_cut = [1989, 1999, 2004, 2009, 2015]
year_range = ['1980-1989','1990-1999','2000-2004','2005-2009','2010-2015','2016-Present']

df_full['year_group'] = df_full.year.apply(lambda x: year_range[len([1 for y in year_cut if y < x])] )

#a = 1990
#year_range[len([1 for x in year_cut if x < a])]

# aggregate by year group
df_year_group = df_full.groupby(['year_group','country','Variable']).agg({'shap':'mean', 'variable value': 'mean'})
df_year_group.reset_index(inplace= True)
df_year_group.rename({'year_group':'year'}, axis= 1, inplace= True)
df_year_group.head()

,year,country,Variable,shap,variable value
0,1980-1989,AE,3-month T-Bill rate,1.536329,8.133542
1,1980-1989,AE,"Bank PD, average",-0.235396,NaN
2,1980-1989,AE,Banking Claims to GDP,-1.891135,NaN
3,1980-1989,AE,Cap Adeq Ratio,-1.022442,NaN
4,1980-1989,AE,"Corp Debt, Sub IG",1.734175,NaN


In [13]:
## append everything
df_full = pd.concat([df_full, df_year_group], axis = 0, sort = True)

In [14]:
df_full = df_full[['year','country','Variable', 'shap','variable value']]

#### calculation by group (to calculate rank on shaply values)

In [15]:
# create rank on shap value (for ranking)
df_full['rank'] = None
df_full.loc[df_full['Variable'] == 'total','rank'] = 30
df_full.loc[df_full['Variable'] == 'base','rank'] = 0

df_full.loc[(df_full['Variable'] != 'base') & (df_full['Variable'] != 'total'),'rank'] = \
df_full.loc[(df_full['Variable'] != 'base') & (df_full['Variable'] != 'total')].groupby(['country','year'])['shap'].rank(ascending= False)

df_full[(df_full['country'] =='China') & (df_full['year']==2018)].sort_values('rank')

# create rank on shap absolute value (for filtering)
df_full['shap_abs'] = abs(df_full.shap)

df_full['rank_abs'] = None
df_full.loc[(df_full['Variable'] != 'base') & (df_full['Variable'] != 'total'),'rank_abs'] = \
df_full.loc[(df_full['Variable'] != 'base') & (df_full['Variable'] != 'total')].groupby(['country','year'])['shap_abs'].rank(ascending= False)

df_full.tail()

,year,country,Variable,shap,variable value,rank,shap_abs,rank_abs
36823,2016-Present,oil-exporter,Real Deposit Growth,-0.517820,-0.118176,18,0.517820,13
36824,2016-Present,oil-exporter,Real Output Growth,0.408381,0.943793,3,0.408381,18
36825,2016-Present,oil-exporter,Total Debt Growth,-0.140766,0.144633,12,0.140766,24
36826,2016-Present,oil-exporter,base,50.000000,NaN,0,50.000000,None
36827,2016-Present,oil-exporter,total,18.302694,NaN,30,18.302694,None


In [16]:
df_full = df_full[['year','country','Variable','shap','variable value','rank','rank_abs']]
df_full.head()

,year,country,Variable,shap,variable value,rank,rank_abs
0,1980,"Afghanistan, I.S. of",3-month T-Bill rate,2.851679,NaN,3,5
1,1981,"Afghanistan, I.S. of",3-month T-Bill rate,2.882570,NaN,2,4
2,1982,"Afghanistan, I.S. of",3-month T-Bill rate,-0.553009,11.100833,19,17
3,1983,"Afghanistan, I.S. of",3-month T-Bill rate,-0.022255,8.944167,17,29
4,1984,"Afghanistan, I.S. of",3-month T-Bill rate,-0.437330,9.897500,20,17
